# Mean-Field AOA

In [ ]:
import numpy as np
import h5py

In [ ]:
import sys
sys.path.append('../src')
from mf_AOA import *

## Sherrington-Kirkpatrick

In [ ]:
# path to folder with the h5 files
PATH_DB = "/home/lappet/Dropbox/MidnightRambler/Mean_Field_AOA/Figures/Data/SK_model/Averaged_MF_energy/"
PATH = "/home/lappet/Archives/projects/QuantumComputing/QAA_and_MFQAOA/data/mean-field-qaoa/results/sherrington-kirkpatrick/new/"

In [ ]:
seed = 137
np.random.seed(seed)

In [ ]:
# schedule
p = 1000
τ = 0.5

γ = τ * (np.arange(1, p + 1) - 1/2) / p
β = τ * (1 - np.arange(1, p + 1) / p)
β[p-1] = τ / (4 * p)

In [ ]:
for num in [10 + k * 5 for k in range(23)]:
    N = num
    num_instances = 10000

    file = "SK_model_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + ".h5"
    print(file)

    h5_file = h5py.File(PATH + file, "w")

    # Sherrington-Kirkpatrick model
    from tqdm import tqdm
    for i in tqdm(range(num_instances)):
        J = np.random.normal(0, 1, size=(N, N)) / np.sqrt(N)
        J = np.triu(J, k=1)
        J = J + J.transpose()

        S = np.array([[1., 0., 0.] for _ in range(N - 1)]) # fix final spin (i.e. leave it out)
        S = evolve(S, J, β, γ)

        h5_file.create_dataset("set_" + str(i + 1) + "/J", data=J)
        h5_file.create_dataset("set_" + str(i + 1) + "/S_z", data=S[:, 2])

    h5_file.close()

In [ ]:
E_0 = -0.763

for num in [10 + k * 5 for k in range(23)]:
    N = num
    num_instances = 10000

    file = "SK_model_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + ".h5"
    print(file)    
    data_file = h5py.File(PATH + file, "r")

    E_star = 0
    E_star_squared = 0
    for i in range(num_instances):
        J = data_file["set_" + str(i + 1) + "/J"][:]
        S_z = data_file["set_" + str(i + 1) + "/S_z"][:]

        E_star += expectation(solution(S_z), J)
        E_star_squared += expectation(solution(S_z), J)**2
        
    out_file = h5py.File(PATH_DB + file, "w")
    out_file.create_dataset("E_star", data=[E_star / num_instances])
    out_file.create_dataset("E_star_squared", data=[E_star_squared / num_instances])
    out_file.close()

In [ ]:
N = 50
num_instances = 10000

file = "SK_model_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + ".h5"
print(file)    
data_file = h5py.File(PATH + file, "r")

E_stars = []
for i in range(num_instances):
    J = data_file["set_" + str(i + 1) + "/J"][:]
    S_z = data_file["set_" + str(i + 1) + "/S_z"][:]

    E_star = expectation(solution(S_z), J)
    E_stars.append(E_star)

file = "SK_model_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + "_all_E_stars.h5"    
out_file = h5py.File(PATH_DB + file, "w")
out_file.create_dataset("all_E_stars", data=E_stars)
out_file.close()